In [1]:
import os,glob
import pandas as pd
import numpy as np

change country between: PR, DR, CU

In [2]:
user = "Chris"
country = "DR"

In [13]:
CU_directory = r"/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/cuba"
DR_directory = r"/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Sync Files + Media"

if user == "Chris":
    if country == "DR":
        current_directory = DR_directory+"/10_28_23_cloud"
        path1 = DR_directory+"/10_28_23_cloud"
        path2 = DR_directory+"/10_28_23_cloud/csv"
    elif country == "CU":
        current_directory = CU_directory+""
        path1 = CU_directory+""
        path2 = CU_directory+""
elif user == "Ty":
    current_directory = r"C:\Users\Ty\Desktop\DataExport"
else:
    print("Unknown user. Cannot set directory.")
    current_directory = None

if current_directory:
    os.chdir(current_directory)
    file_list = os.listdir(current_directory)
    print(f"Changed directory to: {current_directory}")
    print(file_list)

Changed directory to: /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Sync Files + Media/10_28_23_cloud
['Sociodemográfica_202307260149_0_9a4647fcd3952859.epi7', '_informationdoorparticipants_202307260150_0_9a4647fcd3952859.epi7', '_household_202310280820_0_9a4647fcd3952859.epi7', '_informationdoor_202307260149_0_9a4647fcd3952859.epi7', '_mainhousehold_202307260150_0_9a4647fcd3952859.epi7', '.DS_Store', '_rosterhousehold_202307260150_0_9a4647fcd3952859.epi7', '_physical_exam_202307260149_0_9a4647fcd3952859.epi7', 'Neighborhood_202310280820_0_9a4647fcd3952859.epi7', 'Cognitiva_202310280820_0_9a4647fcd3952859.epi7', '_sociodemographic_202310280820_0_9a4647fcd3952859.epi7', '_cognitive_202307260150_0_9a4647fcd3952859.epi7', '_mainhousehold_202310280820_0_9a4647fcd3952859.epi7', 'Listas_202310280820_0_9a4647fcd3952859.epi7', '_nonresidentchildren_202310280820_0_9a4647fcd3952859.epi7', '_household_202307260150_0_9a4647fcd3952859.e

Received files document

In [24]:
# Get the list of all files and directories
dir_list1 = os.listdir(current_directory)

# To see which files I received from the data manager
newdir_list1 = [x for x in dir_list1 if x.endswith('.epi7')]

# To see which files I was able to process with Epi Info tool
dir_list2 = os.listdir(path2)
newdir_list2 = [x for x in dir_list2 if x.endswith('.csv')]

def remove_common(a, b):
    for i in a[:]:
        if i in b:
            a.remove(i)
            b.remove(i)

    print("list1 : ", a)
    print("list2 : ", b)

remove_common(newdir_list1, newdir_list2)

words_to_filter = ["Neighborhood", "Door", "_informationdoor_", "_informationdoorparticipants", "Listas", "_mainhousehold", "_participants", "_rosterhousehold", "_nonresidentchildren", "Cognitva", "_cognitive", "Cognitive_Scoring", "Examen_Físico", "_physical_exam","Familiar", "_household", "Informante", "_informant"]

# breaking the longer list into smaller lists
filtered_lists = {word: [] for word in words_to_filter}

for text in newdir_list2:
    for word in words_to_filter:
        if word in text:
            filtered_lists[word].append(text)

# change this number depending on how many tablets worth of information you're receiving
for word, filtered_list in filtered_lists.items():
    while len(filtered_list) < 1:
        filtered_list.append("Falta")

Received_Files = pd.DataFrame()

for word, filtered_list in filtered_lists.items():
    Received_Files[word] = pd.Series(filtered_list)

Received_Files.to_excel('Received_Files_Checklist.xlsx', index=False)

Received_Files


list1 :  ['_household_202310280820_0_9a4647fcd3952859.epi7', 'Neighborhood_202310280820_0_9a4647fcd3952859.epi7', '_sociodemographic_202310280820_0_9a4647fcd3952859.epi7', '_mainhousehold_202310280820_0_9a4647fcd3952859.epi7', 'Listas_202310280820_0_9a4647fcd3952859.epi7', '_nonresidentchildren_202310280820_0_9a4647fcd3952859.epi7', 'Cognitive_Scoring_202310280819_0_9a4647fcd3952859.epi7', '_rosterhousehold_202310280820_0_9a4647fcd3952859.epi7', 'Examen_Físico_202310280820_0_9a4647fcd3952859.epi7', 'Cognitiva_202310280820_1_9a4647fcd3952859.epi7', 'Door_202310280820_0_9a4647fcd3952859.epi7', '_participants_202310280820_0_9a4647fcd3952859.epi7', 'Informante_202310280819_0_9a4647fcd3952859.epi7', 'Sociodemográfica_202310280820_0_9a4647fcd3952859.epi7', '_informationdoor_202310280820_0_9a4647fcd3952859.epi7', '_physical_exam_202310280820_0_9a4647fcd3952859.epi7', '_informant_202310280820_0_9a4647fcd3952859.epi7']
list2 :  ['_informationdoor_202310280820_0_9a4647fcd3952859.csv', '_inform

,Neighborhood,Door,_informationdoor_,_informationdoorparticipants,Listas,_mainhousehold,_participants,_rosterhousehold,_nonresidentchildren,Cognitva,_cognitive,Cognitive_Scoring,Examen_Físico,_physical_exam,Familiar,_household,Informante,_informant
0,Neighborhood_202310280820_0_9a4647fcd3952859.csv,Door_202310280820_0_9a4647fcd3952859.csv,_informationdoor_202310280820_0_9a4647fcd39528...,_informationdoorparticipants_202307260150_0_9a...,Listas_202310280820_0_9a4647fcd3952859.csv,_mainhousehold_202310280820_0_9a4647fcd3952859...,_participants_202310280820_0_9a4647fcd3952859.csv,_rosterhousehold_202307260150_0_9a4647fcd39528...,_nonresidentchildren_202310280820_0_9a4647fcd3...,Falta,_cognitive_202307260150_0_9a4647fcd3952859.csv,Cognitive_Scoring_202310280819_0_9a4647fcd3952...,Examen_Físico_202310280820_0_9a4647fcd3952859...,_physical_exam_202310280820_0_9a4647fcd3952859...,Familiar_202307260150_0_9a4647fcd3952859.csv,_household_202310280820_0_9a4647fcd3952859.csv,Informante_202310280819_0_9a4647fcd3952859.csv,_informant_202307260150_0_9a4647fcd3952859.csv


In [17]:
folder_path = path2 #goes through this whole folder
dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*_cognitive*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
cog_child = pd.concat(dfs, ignore_index=True)
cog_child = cog_child.drop_duplicates(subset='GlobalRecordId', keep='first')

cog_child.to_csv("CSV_Merged/Cog_Child.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*Cognitiva*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
cog_parent = pd.concat(dfs, ignore_index=True)
cog_parent = cog_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

cog_parent.to_csv("CSV_Merged/Cog_Parent.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*Cognitive_Scoring*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
cog_scoring = pd.concat(dfs, ignore_index=True)
cog_scoring = cog_scoring.drop_duplicates(subset='GlobalRecordId', keep='first')

cog_scoring.to_csv("CSV_Merged/Cog_Scoring.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*_household*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
household_child = pd.concat(dfs, ignore_index=True)
household_child = household_child.drop_duplicates(subset='GlobalRecordId', keep='first')

household_child.to_csv("CSV_Merged/Household_Child.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*Familiar*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
household_parent = pd.concat(dfs, ignore_index=True)
household_parent = household_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

household_parent.to_csv("CSV_Merged/Household_Parent.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*_physical_exam*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
phys_child = pd.concat(dfs, ignore_index=True)
phys_child = phys_child.drop_duplicates(subset='GlobalRecordId', keep='first')

phys_child.to_csv("CSV_Merged/Phys_Child.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*Examen_Físico*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
phys_parent = pd.concat(dfs, ignore_index=True)
phys_parent = phys_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

phys_parent.to_csv("CSV_Merged/Phys_Parent.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*_informant*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
infor_child = pd.concat(dfs, ignore_index=True)
infor_child = infor_child.drop_duplicates(subset='GlobalRecordId', keep='first')

infor_child.to_csv("CSV_Merged/Infor_Child.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*Informante*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
infor_parent = pd.concat(dfs, ignore_index=True)
infor_parent = infor_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

infor_parent.to_csv("CSV_Merged/Infor_Parent.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*_sociodemographic*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
socio_child = pd.concat(dfs, ignore_index=True)
socio_child = socio_child.drop_duplicates(subset='GlobalRecordId', keep='first')

socio_child.to_csv("CSV_Merged/Socio_Child.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*Sociodemográfica*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
socio_parent = pd.concat(dfs, ignore_index=True)
socio_parent = socio_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

socio_parent.to_csv("CSV_Merged/Socio_Parent.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*Neighborhood*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
neighborhood = pd.concat(dfs, ignore_index=True)
neighborhood = neighborhood.drop_duplicates(subset='GlobalRecordId', keep='first')

neighborhood.to_csv("CSV_Merged/Neighborhood.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*Door*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
door_parent = pd.concat(dfs, ignore_index=True)
door_parent = door_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

door_parent.to_csv("CSV_Merged/Door.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*_informationdoor_*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
informationdoor = pd.concat(dfs, ignore_index=True)
informationdoor = informationdoor.drop_duplicates(subset='GlobalRecordId', keep='first')

informationdoor.to_csv("CSV_Merged/InformationDoor.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*_informationdoorparticipants*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
informationdoorparticipants = pd.concat(dfs, ignore_index=True)
informationdoorparticipants = informationdoorparticipants.drop_duplicates(subset='GlobalRecordId', keep='first')

informationdoorparticipants.to_csv("CSV_Merged/InformationDoorParticipants.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*Listas*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
rosters_parent = pd.concat(dfs, ignore_index=True)
rosters_parent = rosters_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

rosters_parent.to_csv("CSV_Merged/Roster_Parent.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*_mainhousehold*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
mainhousehold = pd.concat(dfs, ignore_index=True)
mainhousehold = mainhousehold.drop_duplicates(subset='GlobalRecordId', keep='first')

mainhousehold.to_csv("CSV_Merged/MainHousehold.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*_participants*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
participants = pd.concat(dfs, ignore_index=True)
participants = participants.drop_duplicates(subset='GlobalRecordId', keep='first')

participants.to_csv("CSV_Merged/Participants.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*_rosterhousehold*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
nonparticipants = pd.concat(dfs, ignore_index=True)
nonparticipants = nonparticipants.drop_duplicates(subset='GlobalRecordId', keep='first')

nonparticipants.to_csv("CSV_Merged/NonParticipants.csv")

dfs = [] # list to hold all dataframes

for filename in glob.glob(os.path.join(folder_path, '*_nonresidentchildren*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    dfs.append(df)
    
nonresidentchildren = pd.concat(dfs, ignore_index=True)
nonresidentchildren = nonresidentchildren.drop_duplicates(subset='GlobalRecordId', keep='first')

nonresidentchildren.to_csv("CSV_Merged/NonResidentChildren.csv")

if country != "Cuba":
    dfs = []  # list to hold all dataframes
    for filename in glob.glob(os.path.join(folder_path, '*Sangre*.csv')):
        print(f"Processing {filename}")
        df = pd.read_csv(filename)
        dfs.append(df)
    sangre = pd.concat(dfs, ignore_index=True)
    sangre.to_csv("CSV_Merged/Sangre.csv")

Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Sync Files + Media/10_28_23_cloud/csv/_cognitive_202307260150_0_9a4647fcd3952859.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Sync Files + Media/10_28_23_cloud/csv/Cognitiva_202307260149_0_9a4647fcd3952859.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Sync Files + Media/10_28_23_cloud/csv/Cognitiva_202310280820_1_9a4647fcd3952859.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Sync Files + Media/10_28_23_cloud/csv/Cognitiva_202310280820_0_9a4647fcd3952859.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Sync Files + Media/10_28_23_cloud/csv/Cognitive_Scoring_202310280819

ValueError: No objects to concatenate